<a href="https://colab.research.google.com/github/xuanthang482/CS114.K21/blob/master/Copy_of_CS114_Lab_01_ColabTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CS114 - BÀI TẬP THỰC HÀNH LAB-01
**@Lê Đình Duy**

Đây là tài liệu hướng dẫn thực hành về cách sử dụng Google Colab




# Các thao tác cơ bản với Google Colab

## Các thao tác cơ bản với tập tin Notebook

* Mở tập tin từ GitHub
* Mở tập tin từ Google Drive
* Nạp tập tin từ đĩa cứng
* Lưu tập tin vào GitHub
* Lưu tập tin vào Google Drive
* Lưu tập tin vào đĩa cứng
* Xoá tập tin notebook
* Đổi tên tập tin notebook  
* Xem lịch sử sửa đổi (revision)
* Chia sẻ notebook
* Xem ToC



*Colab notebooks are stored in Google Drive, or can be loaded from GitHub. Colab notebooks can be shared just as you would with Google Docs or Sheets. Simply click the Share button at the top right of any Colab notebook, or follow these Google Drive file sharing instructions.*


## Các thao tác cơ bản với Cell

* Tạo Code cell
* Tạo Text cell 
* Ẩn/hiện code
* Tìm và thêm Code snippets
* Xem các tập tin Files (của máy ảo và Google Drive)
* Thêm form 

## Các thao tác khi thực thi Notebook

* Kết nối với máy thực thi (Hosted runtime vs Local runtime)
* Chế độ Playground
* Thay đổi Runtime type (có dùng GPU, TPU không)
* Thực thi một/nhiều code cell
* Dừng thực thi code cell
* Khởi động lại các thực thi
* Xoá output của một cell
* Xoá all output của notebook
* Quản lý các sessions


*Code is executed in a virtual machine private to your account. Virtual machines are deleted when idle for a while, and have a maximum lifetime enforced by the Colab service.*

*Playground mode opens a copy of the read-only notebook in a temporary session that doesn't save the notebook. If you want to save any changes, you must save the notebook*

## Cài đặt thư viện lên máy thực thi

*To import a library that's not in Colaboratory by default, you can use !pip install or !apt-get install.*

In [0]:
# install 7zip reader
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
import libarchive

In [0]:
# install matplotlib
!pip install -q matplotlib-venn

In [0]:
!apt-get -qq install -y libfluidsynth1

#Các thao tác đọc và hiển thị ảnh dùng OpenCV

## Hiển thị ảnh

In [0]:
from google.colab.patches import cv2_imshow
import cv2

In [0]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
img = cv2.imread('logo.png', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

*Viết lại thành hàm và nhập liệu từ form*


In [0]:
from google.colab.patches import cv2_imshow
import cv2
import os

def download_view(img_url):
  img_file = os.path.basename(img_url)
  !curl -o $img_file $img_url
  img = cv2.imread(img_file, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img)
  return img_file

In [0]:
#@title Nhập URL

img_url = "https://i-vnexpress.vnecdn.net/2020/03/25/my-1-jpeg-2570-1585098851.jpg" #@param {type:"string"}

print(img_url)
img_file = download_view(img_url)
print(img_file)

## Chụp ảnh từ webcam

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

# Các thao tác kết nối với Google Drive để đọc và lưu tập tin

## Kết nối với Google Drive

In [0]:
from google.colab import drive

drive.mount('/content/drive')

*Nạp dữ liệu từ Google Sheet*

In [0]:
!pip install --upgrade -q gspread

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Your spreadsheet name').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

*Đọc file được share*

In [0]:
# ref: https://buomsoo-kim.github.io/colab/2018/04/16/Importing-files-from-Google-Drive-in-Google-Colab.md/
# your code here

*Tải dữ liệu từ Kaggle về Colab*

In [0]:
# ref: https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166
# your code here

*Lưu tập tin vào đĩa cứng cục bộ*

In [0]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Thao tác khác

*Tham khảo Code Snippets hoặc tự đặt câu hỏi và tìm trên Google*

##Play YouTube Video

In [0]:
from IPython.display import YouTubeVideo
YouTubeVideo('IGyA9CxWwbs')

## Play (small size) Video

In [0]:
#Here's a nice way to watch a video within the notebook--will be useful
# when we want to work with videos
import io
import base64
from IPython.display import HTML

def playvideo(filename):
    video = io.open(filename, 'r+b').read()
    encoded = base64.b64encode(video)
    return HTML(data='''<video alt="test" controls>
                    <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
                 </video>'''.format(encoded.decode('ascii')))
#Now play...
playvideo('sample.mp4')

## Javascript to Python Communication

*Python code can register callbacks which can be invoked by Javascript in the outputframe.
Note: This is only available to 'trusted' outputs- outputs which have been executed within the current session*



In [0]:
import IPython
from google.colab import output

def Concat(a, b):
  # Use display.JSON to transfer a structured result.
  return IPython.display.JSON({'result': ' '.join((a, b))})

output.register_callback('notebook.Concat', Concat)

In [0]:
%%javascript
(async function() {
  const result = await google.colab.kernel.invokeFunction(
    'notebook.Concat', // The callback name.
    ['hello', 'world!'], // The arguments.
    {}); // kwargs
  const text = result.data['application/json'];
  document.querySelector("#output-area").appendChild(document.createTextNode(text.result));
})();

In [0]:
import IPython
from google.colab import output

display(IPython.display.HTML('''
    The items:
    <br><ol id="items"></ol>
    <button id='button'>Click to add</button>
    <script>
      document.querySelector('#button').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.AddListItem', [], {});
      };
    </script>
    '''))

def add_list_item():
  # Use redirect_to_element to direct the elements which are being written.
  with output.redirect_to_element('#items'):
    # Use display to add items which will be persisted on notebook reload.
    display(IPython.display.HTML('<li> Another item</li>'))

output.register_callback('notebook.AddListItem', add_list_item)

In [0]:
import IPython
import uuid
from google.colab import output

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def do_something():
  print('here')

InvokeButton('click me', do_something)

## Open files from GitHub

*To open a file from GitHub, you can clone the repo or fetch the file directly.
If you are trying to access files from a private repo, you must use a GitHub access token.*

In [0]:
# Clone the entire repo.
!git clone -l -s git://github.com/jakevdp/PythonDataScienceHandbook.git cloned-repo
%cd cloned-repo
!ls

In [0]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

## Upgrading TensorFlow

*TensorFlow is available by default but you can switch which version you're using.*


In [0]:
# To determine which version you're using:
!pip show tensorflow

# For the current version: 
!pip install --upgrade tensorflow

# For a specific version:
!pip install tensorflow==1.2

# For the latest nightly build:
!pip install tf-nightly